In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import abs, avg, col, length, stddev

spark = SparkSession.builder.master("local[*]").getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 66.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=46aa30ee584257fb02e701e96077b5e944506243909b17bea67e29ca3ce20467
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [4]:
df = spark.read.csv("/content/drive/MyDrive/stocks.csv", inferSchema=True, header=True)
df.describe().show()

+-------+-----------------+-----------------+-----------------+-----------------+-----------------+------+
|summary|             open|             high|              low|            close|           volume|  Name|
+-------+-----------------+-----------------+-----------------+-----------------+-----------------+------+
|  count|           619029|           619032|           619032|           619040|           619040|619040|
|   mean|83.02333431454696|83.77831069346465|82.25609641375786|83.04376276476573|4321823.395568945|  null|
| stddev| 97.3787690433231|  98.207518904464| 96.5074210580914|97.38974800165752|8693609.511967659|  null|
|    min|             1.62|             1.69|              1.5|             1.59|                0|     A|
|    max|           2044.0|          2067.99|          2035.11|           2049.0|        618237630|   ZTS|
+-------+-----------------+-----------------+-----------------+-----------------+-----------------+------+



In [5]:
print(f'DF has { df.count() } records')

DF has 619040 records


In [6]:
appleDF = df.filter(df.Name == "AAPL")
print(f'Apple DF has { appleDF.count() } records')

appleChangesDF = appleDF.withColumn('diff', abs(col('close')-col('open'))).orderBy(col("diff").desc())
print(f'Row with most variation at Apple is { appleChangesDF.take(1) }')

Apple DF has 1259 records
Row with most variation at Apple is [Row(date=datetime.datetime(2015, 8, 24, 0, 0), open=94.87, high=108.8, low=92.0, close=103.12, volume=162206292, Name='AAPL', diff=8.25)]


In [7]:
distinctCompanies = df.select("Name").distinct()
print(f'DF has { distinctCompanies.count() } distinct companies')

letterACompanies = distinctCompanies.filter(distinctCompanies.Name.startswith('A'))
print(f'DF has { letterACompanies.count() } distinct companies starting with A')

n = 2
distinctWithNLetters = distinctCompanies.where(length(col("Name")) == n)
distinctWithNLetters.count()

DF has 505 distinct companies
DF has 59 distinct companies starting with A


50

In [8]:
closeHigherThanOpen = df.filter(df.close > df.open)
print(f'The closing price is higher than the opening in { round((closeHigherThanOpen.count() / df.count()) * 100, 2) }% of the records')

The closing price is higher than the opening in 51.53% of the records


In [9]:
variationDF = df.groupBy("Name").agg(stddev("close")).orderBy(col("stddev_samp(close)").desc())
print(f'Row with most volatile company is { variationDF.take(1) }')

Row with most volatile company is [Row(Name='PCLN', stddev_samp(close)=320.533473018748)]


In [10]:
dateDF = df.groupBy("date").sum("volume").orderBy(col("sum(volume)").desc())
print(f'Row with highest transaction day is { dateDF.take(1) }')

Row with highest transaction day is [Row(date=datetime.datetime(2015, 8, 24, 0, 0), sum(volume)=4607945196)]


In [17]:
mostNegotiationDF = df.groupBy("Name").sum("volume").orderBy(col("sum(volume)").desc())
print(f'Row with most negotiated company is { mostNegotiationDF.take(1) }')

lessNegotiationDF = df.groupBy("Name").sum("volume").orderBy(col("sum(volume)").asc())
print(f'Row with less negotiated company is { lessNegotiationDF.take(1) }')

Row with most negotiated company is [Row(Name='BAC', sum(volume)=117884953591)]
Row with less negotiated company is [Row(Name='APTV', sum(volume)=92947779)]


In [14]:
closeEqualToHighest = df.filter(df.close == df.high)
print(f'The closing price is equal to the highest price in { round((closeEqualToHighest.count() / df.count()) * 100, 2) }% of the records')

The closing price is equal to the highest price in 1.2% of the records
